In [25]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re

import tensorflow_datasets as tfdf
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

In [17]:
path = '/Users/kanishkbhagat/DS/Datasets/NLP/TinyShakespeare.txt'

with open(path) as f:
    text = f.read().lower().split('\n')

In [18]:
def split_punct(text):
    return re.sub(r'([.,?!:])', r' \1', text)

sentence = [split_punct(line) for line in text if line.strip() != '']

In [19]:
tokenizer = Tokenizer(num_words = 5000, filters = '"#$%&()*+-/;<=>@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(sentence)

In [20]:
input_sequence = []
for line in sentence:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        seq = token_list[:i+1]
        input_sequence.append(seq)

In [21]:
input_sequences = np.array(pad_sequences(input_sequence, 
                                maxlen = 20, ))

x, y = input_sequences[:, :-1], input_sequences[:, -1]

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 6)

In [26]:
model = tf.keras.Sequential([
    layers.Input((20,)),
    layers.Embedding(5000, 64),
    
    layers.Bidirectional(layers.LSTM(64, return_sequences = True)),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    
    layers.Bidirectional(layers.LSTM(128, return_sequences = False)),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    
    layers.Dense(5000, activation = 'softmax')
])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 20, 64)         │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 20, 128)        │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 20, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 256)            │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5000)           │     1,285,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,935,752 (7.38 MB)

 Trainable params: 1,934,984 (7.38 MB)

 Non-trainable params: 768 (3.00 KB)

In [34]:
train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).prefetch(tf.data.AUTOTUNE)
val = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(tf.data.AUTOTUNE)

In [36]:
model.compile('adam', 
              loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics = ['accuracy'])

history = model.fit(train, 
                    validation_data = val,
                    epochs = 5)

Epoch 1/5
 167/5105 ━━━━━━━━━━━━━━━━━━━━ 5:30 67ms/step - accuracy: 0.0629 - loss: 8.1146

KeyboardInterrupt: 